In [1]:
from DQNAgent import Experience, ExperienceReplay, Net, DQNAgent
import numpy as np
from time import sleep

In [2]:
from datetime import datetime
import os
dt = datetime.now()
directory = f"LunarLander_results/result-{dt.strftime('%m-%d-%Y_%H-%M-%S')}"
os.mkdir(directory)

In [3]:
import gym
train_dir = directory+'/'+'training'
os.mkdir(directory+'/'+'training')
env = gym.make('LunarLander-v2')
env = gym.wrappers.Monitor(env, train_dir, video_callable=lambda episode_id: episode_id % 200 == 0,force=True)
print(env.action_space)

Discrete(4)


C:\Users\mrida\anaconda3\lib\site-packages\gym\wrappers\monitor.py:31: UserWarning: The Monitor wrapper is being deprecated in favor of gym.wrappers.RecordVideo and gym.wrappers.RecordEpisodeStatistics (see https://github.com/openai/gym/issues/2297)
  warnings.warn(


In [4]:
print(env.action_space.sample())

1


In [5]:
#Change hyperparameters here
agent = DQNAgent(layerSizes=[len(env.reset()), 64, 64, env.action_space.n],
                epsilon=1, eps_decay=0.995,min_eps=0.01, batch_size=64, discount_fact=0.99, 
                 lr=0.0005, num_actions=env.action_space.n, target_update_freq=10000, mem_size = 100000,
                    update_freq = 4, start_training_after=1)

In [6]:
action=0
i_episode = 0
last_n_scores = [0]
progress = []
running = []
scores = []
done = False
n = 100
avg_score_per_n = 0
while np.mean(last_n_scores) < 200:
    
    i_episode += 1
    state = env.reset()
    score = 0
    agent.loss_val = 0
    running = []
    t = 0
    while True:
        t += 1
        action = agent.getAction(state)
        n_state, reward, done, info = env.step(action)
        score += reward
        agent.step(state, action, reward, n_state, int(done))
        running.append(action)
        state = n_state
        
        if done:
#             print(agent.epsilon)
#             print(agent.loss_val/t)
#             print(score)
#             print(f"{i_episode} Episode finished after {t} timesteps")
            if len(last_n_scores) == n:
                last_n_scores.pop(0)
            last_n_scores.append(score)
            if i_episode % 250 == 0:
                progress.append(running)
            break
    avg_score_per_n += score
    if i_episode % n == 0:
        scores.append(avg_score_per_n/n)
        print(agent.epsilon)
        print(f'\nAvg score {i_episode} episode = {avg_score_per_n/n}\n')
        avg_score_per_n = 0
progress.append(running)
env.close()

0.6057704364907278

Avg score 100 episode = -164.86119052335232

0.3669578217261671

Avg score 200 episode = -102.29244210938248

0.22229219984074702

Avg score 300 episode = -59.642546718880084

0.1346580429260134

Avg score 400 episode = -45.52608938408096

0.08157186144027828

Avg score 500 episode = -34.41092177165817

0.0494138221100385

Avg score 600 episode = 1.1895292899316376

0.029933432588273214

Avg score 700 episode = 65.64722135715921

0.018132788524664028

Avg score 800 episode = 135.92339599277588

0.01098430721937979

Avg score 900 episode = 158.11686177594584

0.01

Avg score 1000 episode = 158.4931921728984

0.01

Avg score 1100 episode = 123.270425260567

0.01

Avg score 1200 episode = 155.71889031416208

0.01

Avg score 1300 episode = 169.85845720948095



In [11]:
play_dir = directory+'/'+'play'
# os.mkdir(play_dir)
env = gym.make('LunarLander-v2')
env = gym.wrappers.Monitor(env, play_dir, video_callable=lambda episode_id: True,force=True)

action=0
agent.epsilon = 0
scores = []
time = []
for i_episode in range(10):
    state = env.reset()
    score = 0
    t = 0
    #env is limited to 200 timesteps by default so range(>200 is fine)
    while True:
        t+= 1
        env.render()
        action = agent.getAction(state)
        n_state, reward, done, info = env.step(action)
        sleep(0.008) #Limit to 60 FPS
        state = n_state
        score += reward
        if done:
            scores.append(score)
            print(t)
            time.append(t)
            break
print(f' Mean score = {np.mean(score)}')
print(f' Mean timesteps = {np.mean(time)}')
env.close()

340
313
237
299
365
248
240
184
1000
248
 Mean score = 263.53447515975273
 Mean timesteps = 347.4


In [8]:
##USE THIS CELL TO SAVE TRAINED AGENT

import pickle
from datetime import datetime
dt = datetime.now()

with open(f'{train_dir}/RL-LunarLander-{dt.strftime("%m%d%Y%H%M%S")}.pkl', 'wb') as outp:
    pickle.dump(agent, outp, pickle.HIGHEST_PROTOCOL)

with open(f'{train_dir}/RL-LunarLander-progress{dt.strftime("%m%d%Y%H%M%S")}.pkl', 'wb') as outp:
    pickle.dump(progress, outp, pickle.HIGHEST_PROTOCOL)



In [9]:
1e5

100000.0